In [1]:
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
from scziDesk_preprocess import *
from scziDesk_network import *
from scziDesk_utils import *
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import h5py
import time
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score, calinski_harabasz_score

import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
Using TensorFlow backend.


In [2]:
for category in [ "real_data", "balanced_data", "imbalanced_data",#
                ]:

    path= ".."
    if category in ["balanced_data", "imbalanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files]
    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]
    print(files)

    df = pd.DataFrame(columns = ["dataset", "ARI", "NMI", "sil", "run", "time", "pred", "cal"])
    for dataset in files:
        if category in ["balanced_data", "imbalanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")

        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])
        print(f">>>>dataset {dataset}")
        if X.shape[0] > 10000:
            continue
        n_clusters =  np.unique(Y).shape[0]

        X = np.ceil(X).astype(np.int)
        orig_X = X.copy()
        adata = sc.AnnData(X)
        adata.obs['Group'] = Y

        adata = sc.AnnData(X)
        adata.obs['Group'] = Y
        adata = normalize(adata,
                          copy=True,
                          highly_genes=500,
                          size_factors=True,
                          normalize_input=True,
                          logtrans_input=True)
        X = adata.X.astype(np.float32)
        Y = np.array(adata.obs["Group"])

        for run in range(3):
            start = time.time()
            seed = run
            np.random.seed(seed)
            pca_data = PCA(50, random_state = seed).fit_transform(X)
            kmeans = KMeans(n_clusters=n_clusters, init="k-means++", random_state=seed)
            pred = kmeans.fit_predict(pca_data)

            elapsed = time.time() - start
            ARI = adjusted_rand_score(Y, pred)
            NMI = np.around(normalized_mutual_info_score(Y, pred), 5)
            ss = silhouette_score(pca_data,pred)
            cal = calinski_harabasz_score(pca_data, pred)

            df.loc[df.shape[0]] = [
                dataset, ARI, NMI, ss, run, elapsed, pred, cal
            ]
            df.to_pickle(f"../output/pickle_results/{category}/{category}_pca50_kmeans_scaling.pkl")

['Quake_Smart-seq2_Trachea', 'Quake_Smart-seq2_Diaphragm', 'Quake_10x_Spleen', 'Young', 'mouse_ES_cell', 'Adam', 'Quake_10x_Bladder', 'Quake_Smart-seq2_Lung', 'Quake_10x_Limb_Muscle', 'worm_neuron_cell', 'mouse_bladder_cell', 'Romanov', 'Quake_Smart-seq2_Limb_Muscle', 'Muraro', '10X_PBMC']
>>>>dataset Quake_Smart-seq2_Trachea
>>>>dataset Quake_Smart-seq2_Diaphragm
>>>>dataset Quake_10x_Spleen
>>>>dataset Young
>>>>dataset mouse_ES_cell
>>>>dataset Adam
>>>>dataset Quake_10x_Bladder
>>>>dataset Quake_Smart-seq2_Lung
>>>>dataset Quake_10x_Limb_Muscle
>>>>dataset worm_neuron_cell
>>>>dataset mouse_bladder_cell
>>>>dataset Romanov
>>>>dataset Quake_Smart-seq2_Limb_Muscle
>>>>dataset Muraro
>>>>dataset 10X_PBMC
['data_1c8', 'data_-1c4', 'data_-1c8', 'data_0c4', 'data_0c8', 'data_0c16', 'data_1.5c4', 'data_1c4', 'data_1.5c8', 'data_1.5c16', 'data_-1c16', 'data_1c16']
>>>>dataset data_1c8
>>>>dataset data_-1c4
>>>>dataset data_-1c8
>>>>dataset data_0c4
>>>>dataset data_0c8
>>>>dataset data_0c

In [3]:
df.mean()

ARI      0.337266
NMI      0.363332
sil      0.012638
run      1.000000
time     0.517897
cal     47.846625
dtype: float64

In [ ]:
df.groupby("dataset").mean()